In [1]:
import math
import pickle

import tqdm
import mlflow
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

from utils import *

1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о
составах команд и повопросных результатах (поле mask в results.pkl). Для унификации
предлагаю:

In [2]:
with open('chgk/players.pkl', 'rb') as f:
    players = pickle.load(f)
    
with open('chgk/results.pkl', 'rb') as f:
    results = pickle.load(f)
    
with open('chgk/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)

In [3]:
train_ids = get_ids_by_date([2019], tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [4]:
print(f"Количество турниров для обучения {len(train_ids)}, количество турниров для теста {len(test_ids)}")

Количество турниров для обучения 659, количество турниров для теста 165


In [5]:
# Функции для рассчета коэффициента корреляции Спирмена и Кендалла
# Силу команду определил как среднюю сил участников команды


def get_corr_coef(pred_ratings, actual_ratings, func):
    corrs = []
    for pr, tr in zip(pred_ratings, actual_ratings):
        corrs.append(func(pr, tr).correlation)
    index = ~np.isnan(corrs) * 1
    corrs = np.array(corrs)[index]
    corr_coef = np.mean(corrs)
    return corr_coef

def spearman(pred_ratings, actual_ratings):
    corr_coef = get_corr_coef(pred_ratings, actual_ratings, stats.spearmanr)
    return corr_coef
    
def kendall(pred_ratings, actual_ratings):
    corr_coef = get_corr_coef(pred_ratings, actual_ratings, stats.kendalltau)
    return corr_coef
    
def rating_to_score(ratings):
    positions = []
    sorted_r = sorted(ratings)
    for rating in ratings:
        positions.append(sorted_r.index(rating) + 1)
    return positions

def calc_correlations(ids, results, players_mean_mu):
    pred_ratings = []
    actual_ratings = []
    for id_ in ids:
        tournament = results[id_]
        t_pred_ratings = []
        t_actual_ratings = []
        for team in tournament:
            scores = []
            for member in team['teamMembers']:
                member_id = member['player']['id']
                if member_id in players_mean_mu:
                    scores.append(players_mean_mu[member_id])
            t_actual_ratings.append(team['position'])
            if len(scores):
                index = ~np.isnan(scores)
                scores = list(np.array(scores)[index])
                if len(scores) == 0:
                    print('BAD')
                    t_pred_ratings.append(0)
                else:
                    prob = -np.mean(scores)
                    t_pred_ratings.append(prob)
            else:
                t_pred_ratings.append(0)
        pred_ratings.append(rating_to_score(t_pred_ratings))
        actual_ratings.append(t_actual_ratings)
    
    s_corr = spearman(pred_ratings, actual_ratings)
    k_corr = kendall(pred_ratings, actual_ratings)
    print("Spearman", s_corr)
    print("Kendall", k_corr)
    return s_corr, k_corr

2. Постройте baseline-модель на основе линейной или логистической регрессии, которая
будет обучать рейтинг-лист игроков. 

Проанализировав Ваши замечания и подсказки, я решил (на свой риск и страх) сделать бэйзлайн модель не на основе логистической, либо линейной регрессии, а сделать итерационный алгоритм, правда ничем неподкрепленный, но мне он показался "напрашивающимся".

Так как ответ игрока на какой-либо вопрос $x$ это "верно" либо "неверно", я сделал предположение, что сила какого-либо игрока это параметр $mu_k$ распределения Бернулли, и он зависит от сложности турнира $w_t$:


Вероятность игрока k правильно ответит на вопрос турнира t:

$p(x=1|k, t) = mu_{kt}$

По методу максимального правдоподобия, параметр распределения Бернулли равен отношению количества правильных ответов к количеству всех вопросов: 
        
$mu_{k} = \frac{1}{N}\sum_{i=1}^{N} I(x_i)$

где $I(x_i)$ индикатор, равен 1 если игрок ответил правильно но вопрос $x_i$, и 0 в обратном случае

Но проблема в том, что каждый турнир имеет разную сложность и вот так просто найти силу игрока не получится.
Поэтому я ввел параметр $w_t$ - сложность турнира:

$\sum_{t=1}^{T} w_t = 1$


$mu_{mle}$ для игрока k на турнира t равно ($N_t$ - количество вопросов на турнире t):
        
1. $mu_{kt} = \frac{1}{N_t}\sum_{i=1}^{N_t} I(x_i)$

А вероятность $mu_{k}$ равна (T количество турниров игрока k):
        
2. $mu_{k} = \sum_{t=1}^{T} mu_{kt} * w_{t}$

Возникает вопрос чему равен $w_t$. Например в футболе, для меня лично, сложность турнира определяется количеством сильных команд. Например, лига чемпионов выше уровнем чем лига европы, потому что в 1-турнире участвуют больше грандов.

Поэтому $w_t$ определим как среднюю силу всех игроков, участвовавших на турнире t ($N_p$ - количество участников на турнире t):

3. $w_t = \frac{1}{N_p}\sum_{k=1}^{N_p} mu_{k}$

И так, получилось следующее:
1. Инициализируем $w_t$ случайными значениями от 0 до 1
2. Находим $mu_{k}$ по формуле 2
3. Рассчитываем $w_t$ по формуле 3
4. Повторяем 2-3 шаги до схождения

In [6]:
class BaselineModel:
    def __init__(self, train_ids, results):
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        self.mu_vals = create_nested_dict(train_ids, results)
        
        """Инициализируем сложность турниров случайными значениями"""
        comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        self.consider_questions_num = False
        
        self.train_ids = train_ids
        self.results = results
        
    def calc_players_strength_for_competition(self, comp_id, weight, eps=1e-5):
        """Расчитываем формулу 2 для каждого игрока на турнире comp_id"""
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 0 and len(team['teamMembers']) != 0:
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    new_mu = np.mean(mask)
                    if self.consider_questions_num:
                        penalty = np.sqrt(np.log10(self.player_appereances[player_id] + 1))
                        self.players_mu[player_id].append(new_mu * weight * penalty)                
                    else:
                        self.players_mu[player_id].append(new_mu * weight)
                        
    def get_comp_prob(self, comp_id):
        """Рассчитываем формулу 3 для турнира comp_id"""
        comp = self.mu_vals[comp_id]
        scores = []
        max_mu = 1e-5
        for team in comp:
            for player in comp[team]:
                comp[team][player] = np.mean(self.players_mu[player])
                scores.append(comp[team][player])
                if comp[team][player] > max_mu:
                    max_mu = comp[team][player]
        if len(scores):
            return np.mean(scores), max_mu
        return 1e-5, max_mu
    
    def get_comp_probs(self):
        """Рассчитываем формулу 3 для всех турниров и нормализуем сложность турнира и силу игроков"""
        gl_max_mu = 1e-5
        for comp_id in tqdm.tqdm(self.mu_vals):
            prob, max_mu = self.get_comp_prob(comp_id)
            self.comp_probs[comp_id] = prob
            if max_mu > gl_max_mu:
                gl_max_mu = max_mu
        norm = np.array(list(self.comp_probs.values())).sum()
        for comp_id in self.mu_vals:
            self.comp_probs[comp_id] = self.comp_probs[comp_id] / (norm + 1e-4)
        for player_id in self.players_mu:
            self.players_mu[player_id] = list(np.array(self.players_mu[player_id]) / gl_max_mu)
            self.players_mean_mu[player_id] = np.mean(self.players_mu[player_id])
    
    def iteration(self):
        """Запускаем итерацию 2-3 пунктов"""
        self.players_mu = create_players_dict(self.train_ids, self.results)
        for comp_id in self.comp_probs:
            self.calc_players_strength_for_competition(comp_id, self.comp_probs[comp_id])
        self.get_comp_probs()

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров.
Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь
неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны
эти предсказания сами по себе. Поэтому:

- предложите способ предсказать результаты нового турнира с известными
составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые
корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между
реальным ранжированием в результатах турнира и предсказанным моделью,
усреднённые по тестовому множеству турниров 1 .

Обучим бэйзлайн модель

In [7]:
model = BaselineModel(train_ids, results)
experiment_id = mlflow.set_experiment("baseline-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(30):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:04<00:00, 158.98it/s]


Iteration 1
Spearman 0.6913187305355516
Kendall 0.5263754306893212




100%|██████████| 659/659 [00:04<00:00, 157.97it/s]


Iteration 2
Spearman 0.7550478262366241
Kendall 0.5816433286925994




100%|██████████| 659/659 [00:04<00:00, 157.38it/s]


...

Iteration 28
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:03<00:00, 165.22it/s]


Iteration 29
Spearman 0.7957590406894631
Kendall 0.6188709517282583




100%|██████████| 659/659 [00:03<00:00, 165.06it/s]


Iteration 30
Spearman 0.7957590406894631
Kendall 0.6188709517282583




Видим, что после 22-итерации метрики сошлись и не меняются в последующих итерациях. Взглянем на топ-100 игроков (имя знатока, его сила mu и количество сыгранных вопросов):

In [8]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Роман Петров 1.0 30
2 Игорь Петров 1.0 30
3 Андриан Иоаннидис 1.0 30
4 Дмитрий Луконин 1.0 30
5 Анатолий Королихин 1.0 30
6 Евгений Чернецкий 0.8888888888888888 30
7 Александр Усов 0.8888888888888888 30
8 Дамир Фёдоров 0.8888888888888888 30
9 Василий Марков 0.8888888888888888 30
10 Константин Карпов 0.6111111111111112 30
11 Иван Карпов 0.6111111111111112 30
12 Виктория Кулёва 0.6111111111111112 30
13 Яна Кулёва 0.6111111111111112 30
14 Юлия Крюкова 0.055985488413390794 36
15 Наталья Артемьева 0.055985488413390794 36
16 Екатерина Горелова 0.055985488413390794 36
17 Антон Калинин 0.054235941900472345 36
18 Ольга Остросаблина 0.05170086896016139 36
19 Валентина Подюкова 0.05090567159250791 36
20 Махбуба Мамаджанова 0.050736848874635414 36
21 Дарина Калнина 0.04898730236171695 36
22 Артём Улюкин 0.04898730236171695 36
23 Мария Каменских 0.04819694046419397 72
24 Сергей Пашкевич 0.04732111838263656 36
25 Анна Щеголькова 0.04548820933588003 36
26 Матвей Конюхов 0.04548820933588003 36
27 Ив

В топе рейтинга игроков действительно ноунеймы, сыгравшие мало вопросов. Также в рейтинге есть игроки и с большим количеством сыгранных вопросов, и вроде бы все они действительно топы. А теперь взглянем на топ-100 турниров по сложности

In [9]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Чемпионат Кипра среди школьников 0.06121244354286861
2 Чемпионат Мира. Финал. Группа А 0.0025715075836457357
3 Чемпионат Мира. Этап 2. Группа А 0.002548336397093613
4 Чемпионат Мира. Этап 3. Группа А 0.0025443062459037037
5 Чемпионат Мира. Этап 1. Группа А 0.0023566904776438397
6 Чемпионат Санкт-Петербурга. Высшая лига 0.002178197186366745
7 Чемпионат России 0.0021496788713585265
8 Чемпионат Мира. Финал. Группа В 0.0021205941823524603
9 Чемпионат Мира. Этап 3. Группа В 0.0021155542521772326
10 Славянка 0.0021084320671363893
11 Чемпионат Мира. Этап 2. Группа В 0.0020994192913743647
12 Весна в ЛЭТИ 0.0020330833879260166
13 Мемориал Дмитрия Коноваленко 0.002025390868654653
14 Второй тематический турнир имени Джоуи Триббиани 0.002023272262609647
15 Мемориал Дмитрия Мотрича 0.002016794942869896
16 Год театра 0.002011715507763516
17 Чемпионат Мира. Этап 1. Группа В 0.0019958166752526013
18 Регулярный чемпионат МГУ. Высшая лига. Третий игровой день 0.0019929889191269484
19 Знатокиада. Всеоб

На первом месте чемпионат среди школьников, видимо те ноунеймы и подняли рейтинг этого турнира, далее идут разные этапы чемпионата мира и чемпионат России, вроде более-менее правильно

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько
игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать
упрощающие предположения, но теперь переменные “игрок X ответил на вопрос
Y” при условии данных должны стать зависимыми для игроков одной и той же
команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем
растут (скорее всего, ненамного, но расти должны), выберите лучшую модель,
используя целевые метрики.

Рассмотрим правдоподобие результата $X$ какой-либо команды $m$ на каком-либо турнире $t$ если в команде только один игрок:

$p(X_{tm}|mu_m, t) = \prod_{i=1}^{N} mu_m^{x_i} * (1 - mu_m) ^ {1 - x_i}$

Но в командах обычно больше чем один игрок. Нам неизвестно кто из игроков ответил на определенный вопрос. Вообще говоря бывают разные случаи: бывает все игроки знают ответ, бывает несколько игроков знают и отвечают правильно, бывает несколько знатоков имеют правильную версию, но отвечает игрок с неправильной версией. Смоделировать эти ситуации сложно. Я решил упростить задачу и сделать предположение, что если команда ответила неправильно, то ни у кого не было правильной версии. Дальше я сделал предположение, что на отвеченный вопрос знал ответ только один знаток. Поэтому у меня получилась некая класстеризация вопросов по игрокам. Давайте рассмотрим правдоподобие результата (K количество игроков в команде m, $pi_k$ - равно 1 если k-ый игрок ответил на вопрос, и 0 если нет):

$p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}) = \prod_{i=1}^{N} (\sum_{k=1}^{K} pi_k * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})$

$\sum_{k=1}^{K} pi_k = 1$

Рассмотрим логарифм правдоподобия:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, pi)) = \sum_{i=1}^{N} (log(\sum_{k=1}^{K} pi_{mk} * mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i}))$

И как мы видим максимизировать это правдоподобие сложно, поэтому внесем скрытые переменные $z$:

$log(p(X_{tm}|mu_{m1}, mu_{m2} ... mu_{mk}, z)) = \sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

$\sum_{k=1}^{K} z_k = 1$

Теперь распишем правдободобие для всех данных (все турниры T, все команды M, все вопросы N):

$log(X|everything) = \sum_{t=1}^{T}\sum_{m=1}^{M}\sum_{i=1}^{N} (log(\prod_{k=1}^{K} (mu_{mk}^{x_i} * (1 - mu_{mk}) ^ {1 - x_i})^ {z_{mk}}))$

Найти ожидание $z_{mkt}$ можно по формуле:

1. $E[z_{mkt}] = \frac{pi_{mkt} * p(X_{tm}|mu_{mk}, t)}{\sum_{j=1}^{K}pi_{mjt} * p(X_{tm}|mu_{mj}, t)} $

Опять же тут у всех турниров разные уровни, поэтому я ввел следующее равенство, вероятность игрока k ответит на вопрос турнира t равно (p_k - сила игрока, w_t - сложность вопроса):

2. $p(x=1|k, t) = \frac{p_k}{p_k + w_t} = mu_{kt}$


Теперь приступим к EM-алгоритму:

- на шаге expectation найдем все $E[z_{mkt}]$
- на шаге максимизации нужно найти значения $pi_{mkt}$, $w_t$, $p_k$. $mu_{kt}$ найдем по формуле 2


9. $pi_{kt} = \frac{1}{N}\sum_{n=1}^{N}E[z_{nkt}]$   (формулы 9.60 из Bishop)

А для нахождения $p_k$ и $w_t$ распишем часть правдободобия, где присутсвуют эти параметры:

$l = \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} * log(\frac{p}{p + w_t}^{x_n} * (1 - \frac{p}{p + w_t})^{1 - x_n}) + ...$

$l = \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} * log(\frac{p_k}{p_k + w}^{x_n} * (1 - \frac{p_k}{p_k + w})^{1 - x_n}) + ...$

Найдем производную по $p$ и попытаемся найти экстремум функции:

3. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \sum_{t=1}^{T}\sum_{n=1}^{N_t} \frac{z_{nt}}{p + w_t} = 0$

4. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \sum_{k=1}^{K}\sum_{n=1}^{N}\frac{z_{nk}}{p_k + w} = 0$

Найти значения $p$ и $w$ сложно, поэтому я использовал scipy.optimize.fsolve

###### Отступление:
Изначально я посчитал формулы 3 и 4 неправильно, как:

5. $\frac{dl}{dp} = \frac{1}{p} \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n - \frac{1}{p + w_t}\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt}  = 0$

6. $\frac{dl}{dw} = \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} - \frac{1}{w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk}x_n - \frac{1}{p_k + w}\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} = 0$

И естественно легко максимизировал их, и получил:

7. $p = \frac{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n w_t}{\sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} - \sum_{t=1}^{T}\sum_{n=1}^{N_t}z_{nt} x_n}$


8. $w = \frac{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k - \sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} p_k x_n}{\sum_{k=1}^{K}\sum_{n=1}^{N}z_{nk} x_n}$

По формуле 7 было видно, что чем больше отвеченных вопросов у знатока и чем больше сложность турнира, тем больше его сила, и наоборот. А по формуле 8 было заметно, что чем больше неправильных ответов у сильного знатока, тем больше сложность вопроса, все вроде бы логично. НО при расчете этих формул была допущена грубая ошибка, которую я заметил только в последний день дедлайна, когда начал писать чистовую версию. Но эти значения все же использую как estimated value для scipy.optimize.fsolve

In [10]:
def dl_dmu(mu, z_x, znts, weights):
    """правдоподобие по формуле 5"""
    temp = z_x / mu
    for i, znt in enumerate(znts):
        temp -= znt / (weights[i] + mu)
    return temp

In [11]:
def dl_dw(weight, znk, z_x, mus):
    """правдоподобие по формуле 6"""
    result = (np.sum(znk) - np.sum(z_x)) / weight
    for mu, z in zip(mus, znk):
        result -= z / (weight + mu)
    return result

In [20]:
class EMModel:
    def __init__(self, train_ids, results, q=1.5, force=False, cut_num=None):
        self.z_vals = create_nested_dict(train_ids, results)
        self.pi_vals = create_nested_dict(train_ids, results)
        self.player_appereances = get_players_appereances(train_ids, results)
        self.players_mean_mu = create_players_dict(train_ids, results, True)
        # comp_probs = np.random.uniform(low=1.0, high=2.0, size=len(train_ids))
        comp_probs = np.ones(len(train_ids))
        self.comp_probs = {id_: prob for id_, prob in zip(train_ids, comp_probs)}
        
        self.train_ids = train_ids
        self.results = results
        self.q = q
        self.force = force
        self.cut_num = cut_num
        
    def calculate(self, func, eps=1e-5):
        for comp_id in tqdm.tqdm(self.comp_probs):
            weight = self.comp_probs[comp_id]
            for team in self.results[comp_id]:
                team_id = team['team']['id']
                mask = team['mask']
                mask = to_int(mask)
                func(team, mask, weight, comp_id, team_id, eps)
        
    def expectation(self, team, mask, weight, comp_id, team_id, eps):
        """Находим ожидание скрытых пременных"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            pis = []
            mus = []
            # Initialize
            for player in team['teamMembers']:
                player_id = player['player']['id']
                pis.append(self.pi_vals[comp_id][team_id][player_id])
                mus.append(self.players_mean_mu[player_id] / (self.q * self.players_mean_mu[player_id] + weight + eps))
                self.z_vals[comp_id][team_id][player_id] = []

            pis = np.array(pis)
            mus = np.array(mus)

            # Expectation
            for answer in mask:
                denominator = pis * (mus ** answer) * (1 - mus) ** (1 - answer)
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    znk = denominator[i] / (np.sum(denominator) + eps)
                    assert znk <= 1, denominator
                    self.z_vals[comp_id][team_id][player_id].append(znk)

    def maxizmize_mus_and_pis_by_team(self, team, mask, weight, comp_id, team_id, eps):
        """Максимизируем pi по формуле 9"""
        if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
            # Maximization
            for i, player in enumerate(team['teamMembers']):
                player_id = player['player']['id']
                nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                new_mu = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                self.z_x[player_id].append(new_mu)
                self.znt[player_id].append(nk)
                self.w_t[player_id].append(weight)
                self.pi_vals[comp_id][team_id][player_id] = nk / (mask.shape[0] + eps)

    def maximize_player_mus(self):
        """Максимизируем p по формуле 5"""
        for player_id in self.z_x:
            if self.cut_num:
                if len(self.z_x[player_id]) > self.cut_num:
                    self.z_x[player_id] = self.z_x[player_id][-self.cut_num:]
                if len(self.w_t[player_id]) > self.cut_num:
                    self.w_t[player_id] = self.w_t[player_id][-self.cut_num:]
                if len(self.znt[player_id]) > self.cut_num:
                    self.znt[player_id] = self.znt[player_id][-self.cut_num:]
                
            z_x = np.sum(self.z_x[player_id])
            mu = np.sum(np.array(self.z_x[player_id]) * np.array(self.w_t[player_id]))
            estimate_mu = mu / (np.sum(self.znt[player_id]) - np.sum(self.z_x[player_id]))
            new_mu = fsolve(dl_dmu, estimate_mu, args=(z_x, self.znt[player_id], self.w_t[player_id]), xtol=1)[0]
            if self.force:
                k = np.log10(self.player_appereances[player_id])
                k = pow(k, 0.25)
                new_mu = new_mu * k
            self.players_mean_mu[player_id] = new_mu
            assert new_mu >= 0, (new_mu, mu, z_x, self.znt[player_id], self.w_t[player_id])

    def maxizmize_comp_weights(self, comp_id, eps=1e-5):
        """Максимизируем w по формуле 6"""
        znk_xn = []
        znk = []
        mus = []
        for team in self.results[comp_id]:
            team_id = team['team']['id']
            mask = team['mask']
            mask = to_int(mask)
            if mask.shape[0] > 1 and len(team['teamMembers']) != 0:
                # Maximization
                for i, player in enumerate(team['teamMembers']):
                    player_id = player['player']['id']
                    nk = np.sum(self.z_vals[comp_id][team_id][player_id]) + eps
                    summ = np.sum(mask * np.array(self.z_vals[comp_id][team_id][player_id]))
                    znk_xn.append(summ)
                    znk.append(nk)
                    mus.append(self.players_mean_mu[player_id])
        
        znk_uk_sum = np.sum(np.array(znk) * np.array(mus))
        znk_xn_uk_sum = np.sum(np.array(znk_xn) * np.array(mus))
        znk_xn_sum = np.sum(znk_xn)
        estimate_weight = (znk_uk_sum - znk_xn_uk_sum) / (znk_xn_sum + eps)
        self.comp_probs[comp_id] = fsolve(dl_dw, estimate_weight, args=(znk, znk_xn, mus), xtol=1)[0]
        assert self.comp_probs[comp_id] >= 0, self.comp_probs[comp_id]
            
    def maxizmize_comps_weights(self, eps=1e-5):
        for comp_id in self.comp_probs:
            self.maxizmize_comp_weights(comp_id)
            
    def iteration(self):
        self.z_x = create_players_dict(self.train_ids, self.results)
        self.znt = create_players_dict(self.train_ids, self.results)
        self.w_t = create_players_dict(self.train_ids, self.results)
        self.calculate(self.expectation)
        self.calculate(self.maxizmize_mus_and_pis_by_team)
        self.maximize_player_mus()
        self.maxizmize_comps_weights()

Обучим нашу модель:

In [13]:
model = EMModel(train_ids, results, q=1)
experiment_id = mlflow.set_experiment("EM-model")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(5):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 80.11it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/scipy/optimize/minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Iteration 1
Spearman 0.6958132465351412
Kendall 0.5259170823109763




100%|██████████| 659/659 [00:08<00:00, 75.52it/s] 


Iteration 2
Spearman 0.7499631253046996
Kendall 0.5774678115487147




100%|██████████| 659/659 [00:08<00:00, 80.38it/s] 


Iteration 3
Spearman 0.7568632485373844
Kendall 0.5856577954917372




100%|██████████| 659/659 [00:08<00:00, 79.75it/s] 


Iteration 4
Spearman 0.75899049970781
Kendall 0.5885989126215737




100%|██████████| 659/659 [00:08<00:00, 76.87it/s] 


Iteration 5
Spearman 0.7574463839049668
Kendall 0.5861672101454283




По традиции посмотрим на топов:

In [14]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 София Савенко 5235.83252993919 36
2 Елена Бровченко 323.1827414310536 36
3 София Лебедева 320.50872855899337 36
4 Андрей Зацаринный 219.75074233492046 36
5 Александр Корнюков 163.23572227634205 80
6 Полина Кошелева 139.68612283789525 36
7 Валентина Подюкова 137.9894342667877 36
8 Максим Пилипенко 113.39800334594642 36
9 Семён Стенюгин 106.2936442837909 36
10 Семён Зайдельман 105.3761094322556 36
11 Екатерина Золотухина 103.08130249577482 36
12 Полина Максимов 99.22529082118659 36
13 Пётр Беляев 95.18591739285796 36
14 Артём Стетой 93.49193513901994 36
15 Глеб Гаврилов 80.45433066332716 36
16 Лейсан Чхеидзе 73.85515828505068 36
17 Елизавета Коваленко 66.24208077669101 36
18 Георгий Титов 63.39368846239751 36
19 Юлия Карпук 62.48056842427467 36
20 Диана Папян 60.74350732280858 36
21 Виталий Сухинин 60.61737356069423 36
22 Полина Джегур 59.27833236987754 36
23 Дмитрий Ичко 59.04510727473508 36
24 Надежда Бирюкова 56.826594619110345 36
25 Екатерина Лукьянцева 56.17029630317944 72
26 Елиз

Видим, что топ полностью забит ноунемами с маленьким количеством сыгранных вопросов

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов.
Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть
сложные вопросы, а на турнирах для школьников — простые)?

In [15]:
top_comps = get_top_comps(model.comp_probs, 100, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 6.199571602915363
2 Синхрон высшей лиги Москвы 4.861172598790621
3 Воображаемый музей 4.446339907916658
4 Первенство правого полушария 4.3277198989279615
5 Записки охотника 3.9020752160692482
6 Ускользающая сова 3.8176764079006906
7 Знание – Сила VI 3.6639963908630384
8 Зеркало мемориала памяти Михаила Басса 3.6078916952950353
9 Чемпионат Мира. Этап 2. Группа В 3.463343835420427
10 Чемпионат Минска. Лига А. Тур четвёртый 3.4256636931743905
11 Чемпионат России 3.3697082156929485
12 Чемпионат Мира. Этап 2. Группа А 3.352692749623997
13 Кубок городов 3.3458863268744365
14 Чемпионат Мира. Этап 2 Группа С 3.2602629932973723
15 Чемпионат Мира. Этап 3. Группа В 3.2183785330018995
16 Львов зимой. Адвокат 3.1759413232256417
17 Чемпионат Мира. Финал. Группа С 3.1148942492074583
18 Серия Premier. Седьмая печать 3.1148336810701984
19 Тихий Донец: омут первый 3.052598004167396
20 All Cats Are Beautiful 3.050110253388775
21 Антибинго 3.018649239272013
22 VERSUS: Коробейников vs. Матвее

###### В топ-20 много этапов чемпионата мира, есть чемпионат России, школьных турниров не вижу, поэтому можно сказать, что рейтинг более-менее соответствует реальности

## 6
Как мы видим в топе игроки, которые сыграли мало игр, попытаемся это исправить.

Для этого придумал 2 подхода:

1. В формулу 2 добавим параметр q > 1 (я назвал его параметром регуляризации) 

$p(x=1|k, t) = \frac{p_k}{q * p_k + w_t} = mu_{kt}$

Как мы видим этот параметр препятствует игроку стать слишком "сильным", поэтому это должно сместить игроков ставших сильными засчет малого количества игр

2. Умножить силу игрока на функцию от количества сыгранных вопросов. Но это конечно явное притягивание за уши


- k = np.log10(questions_num)
- k = pow(k, 0.25)
- new_p = p * k

In [16]:
# Попробуем вариант 1
model = EMModel(train_ids, results, q=1.5)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 78.89it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7142535390847692
Kendall 0.5460150558762311




100%|██████████| 659/659 [00:08<00:00, 80.40it/s] 


Iteration 2
Spearman 0.7856042345299682
Kendall 0.6087757375318611




100%|██████████| 659/659 [00:08<00:00, 78.89it/s] 


...

Iteration 13
Spearman 0.8220881656250344
Kendall 0.6476939634597919




100%|██████████| 659/659 [00:08<00:00, 79.71it/s] 


Iteration 14
Spearman 0.8236592331273218
Kendall 0.6495962112993748




100%|██████████| 659/659 [00:08<00:00, 79.46it/s] 


Iteration 15
Spearman 0.8237777874163525
Kendall 0.6495387085236644




In [17]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Максим Пилипенко 9.162648190943568 36
2 Максим Руссо 6.849129919736287 2178
3 Иван Семушин 6.538804076707349 3774
4 Дмитрий Кудинов 6.401973885316748 45
5 Мария Каменских 6.276560069051914 72
6 Михаил Савченков 6.2376843815892835 3215
7 Валентина Подюкова 6.178045594453892 36
8 Сусанна Бровер 5.939703451670819 600
9 Станислав Мереминский 5.8656380038323 1584
10 Артём Сорожкин 5.80672065735904 4849
11 Дмитрий Карякин 5.581549655397248 1304
12 Александра Брутер 5.560105561549427 2692
13 Алексей Гилёв 5.343473605928458 4306
14 Анна Карпелевич 5.249494193492643 60
15 Владимир Самойлов 5.246353504201363 115
16 Игорь Мокин 5.244520899411299 1176
17 Сергей Завьялов 5.217804729416079 36
18 Анастасия Калинина 5.152641996409019 36
19 Александр Либер 5.122783680713623 3751
20 Михаил Левандовский 5.094334664805087 1457
21 Александр Коробейников 5.076365676912233 1401
22 Денис Петров 5.046538223788496 72
23 Сергей Спешков 5.036784927359708 3737
24 Евгений Дёмин 5.024217705741187 537
25 Евгений Сп

###### Действительно, топов стало намного больше в рейтинге. Регуляризация помогает.

In [18]:
top_comps = get_top_comps(model.comp_probs, 20, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 6.192714739842387
2 Синхрон высшей лиги Москвы 5.099937895026505
3 Первенство правого полушария 4.557415946033383
4 Воображаемый музей 4.387580772096771
5 Записки охотника 4.055957050476376
6 Ускользающая сова 4.036193380379926
7 Знание – Сила VI 3.9493639217971954
8 Чемпионат Мира. Этап 2 Группа С 3.670861136880585
9 Чемпионат Минска. Лига А. Тур четвёртый 3.6345870137387735
10 Чемпионат Мира. Этап 2. Группа В 3.593224477909376
11 Зеркало мемориала памяти Михаила Басса 3.500234004790419
12 Кубок городов 3.5001668039391753
13 Чемпионат России 3.461669342879435
14 Чемпионат Мира. Этап 3. Группа С 3.3143617723624894
15 Чемпионат Мира. Финал. Группа С 3.308526500114064
16 Чемпионат Мира. Этап 3. Группа В 3.2779122668400715
17 Тихий Донец: омут первый 3.231981089764124
18 Чемпионат Мира. Этап 1. Группа С 3.1961818315900414
19 Чемпионат Мира. Этап 2. Группа А 3.176981445618495
20 All Cats Are Beautiful 3.165341203047863


###### Рейтинг турниров вроде бы особо не изменился

In [21]:
# Попробуем вариант 2

# force - bool, усилять ли силу игрока в зависимости от количества сыгранных вопросов или нет


model = EMModel(train_ids, results, q=1.5, force=True)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 659/659 [00:08<00:00, 78.72it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7319043678073328
Kendall 0.5601984603028859




100%|██████████| 659/659 [00:08<00:00, 79.39it/s] 


Iteration 2
Spearman 0.8017055447492656
Kendall 0.6240205619726511




100%|██████████| 659/659 [00:08<00:00, 79.95it/s] 


...

Iteration 13
Spearman 0.8224332370294613
Kendall 0.6496485861611725




100%|██████████| 659/659 [00:08<00:00, 80.02it/s] 


Iteration 14
Spearman 0.8224669689898705
Kendall 0.6500302286946059




100%|██████████| 659/659 [00:08<00:00, 79.92it/s] 


Iteration 15
Spearman 0.8225376182638208
Kendall 0.6500847490565246




In [22]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Максим Руссо 451.1931390320189 2178
2 Евгений Спектор 389.80495273676485 230
3 Михаил Савченков 350.8067715344045 3215
4 Иван Семушин 347.266588062803 3774
5 Станислав Мереминский 345.9745833539963 1584
6 Артём Сорожкин 338.0524046362004 4849
7 Александра Брутер 331.2163022954075 2692
8 Сусанна Бровер 314.2509856806073 600
9 Алексей Гилёв 311.04156292229067 4306
10 Максим Пилипенко 309.99526744163 36
11 Дмитрий Кудинов 309.5219963235307 45
12 Александр Либер 306.66678834733284 3751
13 Ирина Прокофьева 296.1647495168821 1065
14 Михаил Царёв 294.4546447674751 501
15 Александр Марков 292.65829684576767 2903
16 Ирина Пионтковская 291.92538485642393 67
17 Игорь Мокин 290.3974230149984 1176
18 Михаил Левандовский 289.43188728537854 1457
19 Сергей Спешков 289.35846226292654 3737
20 Дмитрий Карякин 283.61651410322696 1304
21 Наталья Кудряшова 280.762343243675 2590
22 Александр Коробейников 275.05162055619974 1401
23 Андрей Островский 274.93176642018943 2612
24 Евгений Дёмин 271.4594688272736

###### Вариант 2 тоже помог и топов стало еще больше в рейтинге

In [23]:
top_comps = get_top_comps(model.comp_probs, 20, tournaments)
for i, comp in enumerate(top_comps):
    print(i + 1, comp[0], comp[1])

1 Угрюмый Ёрш 357.81277104338767
2 Синхрон высшей лиги Москвы 287.7669266773632
3 Воображаемый музей 260.8074697756976
4 Первенство правого полушария 258.12895586689496
5 Записки охотника 230.7386105162411
6 Ускользающая сова 226.87997917828994
7 Знание – Сила VI 222.4382854206179
8 Чемпионат Минска. Лига А. Тур четвёртый 199.33786287470505
9 Чемпионат Мира. Этап 2 Группа С 198.94295898361972
10 Чемпионат Мира. Этап 2. Группа В 198.68486215087756
11 Чемпионат России 198.44731133062558
12 Зеркало мемориала памяти Михаила Басса 195.7535367207241
13 Кубок городов 189.64531294023564
14 Чемпионат Мира. Этап 2. Группа А 189.2537873653787
15 Тихий Донец: омут первый 186.95658272519302
16 Чемпионат Мира. Этап 3. Группа С 186.80259486750612
17 Львов зимой. Адвокат 186.65843085479455
18 Чемпионат Мира. Этап 3. Группа В 184.72717083209295
19 Серия Premier. Седьмая печать 184.3597288765971
20 Чемпионат Мира. Финал. Группа С 184.1898358369643


И коэффициент q и выражение силы игрока через функцию сыгранных им вопросов помогает сделать рейтинг игроков лучше. Но второй вариант работает за 1 год, а за 10 лет может совсем не сработать. Поэтому дальше буду использовать только коэффициент q (q=1.5)

## 7

Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей
модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг
вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь
возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем
этого добиться:
2
- если хватит вычислительных ресурсов, сначала сделайте baseline совсем без
таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не
только на турнирах 2019 года; улучшилось ли качество предсказаний на 2020?
- одну схему со временем мы уже использовали: брали для обучения только
последний год турниров; примерно так делают, например, в теннисной
чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки
(например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
- предложите варианты базовой модели или алгоритма её обучения, которые
могли бы реализовать изменения рейтинга со временем; если получится,
реализуйте их на практике, проверьте, улучшатся ли предсказания на 2020.

Обучим модель с 2010 года. Модель использовал с тем же параметром q, но без домножения на количество сыгранных вопросов, поэтому качество будем сверять с качеством предпоследней модели

In [24]:
train_ids = get_ids_by_date(list(range(2010, 2020)), tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [25]:
model = EMModel(train_ids, results, q=1.5, force=False)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force-from2010")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 3328/3328 [00:39<00:00, 83.35it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7106410480343532
Kendall 0.5394182532202937




100%|██████████| 3328/3328 [00:40<00:00, 82.43it/s] 


Iteration 2
Spearman 0.7851167444009398
Kendall 0.6037893554159199




100%|██████████| 3328/3328 [00:39<00:00, 84.59it/s] 


...

Iteration 13
Spearman 0.8116261576348835
Kendall 0.6398438911611146




100%|██████████| 3328/3328 [00:38<00:00, 85.74it/s] 


Iteration 14
Spearman 0.8122663556835448
Kendall 0.6406071660238397




100%|██████████| 3328/3328 [00:38<00:00, 86.16it/s] 


Iteration 15
Spearman 0.8127110833532472
Kendall 0.640940248649734




Аналогичная модель, обученная только на данных 2019 года показала результат по Спирмену 0.8237, по Кендаллу 0.6495, что лучше чем результат модели, обученной на данных с 2010 года.

###### Возможно лучше брать последние N (например 50) турниров за последние 2 или 3 года.

In [26]:
train_ids = get_ids_by_date(list(range(2015, 2020)), tournaments, results)
test_ids = get_ids_by_date([2020], tournaments, results)

In [29]:
# cut_num - рассматривать только последние cut_num турниров, если None рассматриваем все турниры

model = EMModel(train_ids, results, q=1.5, force=False, cut_num=50)
experiment_id = mlflow.set_experiment("EM-model-with-q-regularization-and-force-and_cut_from2010")
with mlflow.start_run(experiment_id=experiment_id):
    for i in range(15):
        model.iteration()
        print('Iteration', i + 1)
        s_corr, k_corr = calc_correlations(test_ids, results, model.players_mean_mu)
        mlflow.log_metric("Spearman", s_corr, i + 1)
        mlflow.log_metric("Kendall", k_corr, i + 1)
        print('\n')

100%|██████████| 2575/2575 [00:31<00:00, 81.72it/s] 
/home/azamat/miniconda3/envs/conda_work/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


Iteration 1
Spearman 0.7691567976073015
Kendall 0.5940239024126795




100%|██████████| 2575/2575 [00:31<00:00, 80.53it/s] 


Iteration 2
Spearman 0.8118836615289411
Kendall 0.6355759738547322




100%|██████████| 2575/2575 [00:32<00:00, 80.26it/s] 


...

Iteration 13
Spearman 0.8267376420035353
Kendall 0.6547562650779893




100%|██████████| 2575/2575 [00:31<00:00, 82.86it/s] 


Iteration 14
Spearman 0.8272969802223112
Kendall 0.6551379025093521




100%|██████████| 2575/2575 [00:32<00:00, 79.31it/s] 


Iteration 15
Spearman 0.8278660981865531
Kendall 0.6557981029335572




Текущая версия показывает метрики Спирман-0.8278, Кендалл-0.656, что лучше модели, обученной на данных 2019 года. Но вот рейтинг игроков кажется стал хуже:

In [30]:
top_players = get_top_players(model.players_mean_mu, 100, players, model.player_appereances)
for i, player in enumerate(top_players):
    print(i + 1, player[0], player[1], player[2])

1 Илья Леошкевич 9.405290671342382 30
2 Евгений Поникаров 9.335919557225239 36
3 Евгений Аренгауз 8.848883308944535 45
4 Максим Руссо 8.35971204383234 12816
5 Владислав Гребцов 7.967578046336044 43
6 Александр Аникин 7.087619964592252 42
7 Валентина Подюкова 6.494374313357586 36
8 Ирина Прокофьева 6.099737012948676 5216
9 Артём Богданович 5.952140994485605 36
10 Иван Семушин 5.887967787675137 18139
11 Галина Тепфер 5.851547963120102 356
12 Сергей Спешков 5.7864912490685345 13347
13 Константин Кноп 5.736721606834526 201
14 Мария Летюхина 5.53287744842358 2317
15 Артём Сорожкин 5.426139595200335 19156
16 Антон Губанов 5.401084287178812 334
17 Владимир Брайман 5.366317367568913 307
18 Евгения Шатохина 5.346137037793719 36
19 Анна Карпелевич 5.3152579184089 60
20 Станислав Мереминский 5.291030366140891 12420
21 Михаил Левандовский 5.270694767423843 7646
22 Александра Брутер 5.255964669485728 16289
23 Дмитрий Фрадис 5.244611640305782 403
24 Лада Латышева 5.217962302294169 36
25 Михаил Савче

Подводя итоги, метод, где учитываются только последние N турниров, за последние M лет, помог сделать ранжирование команд лучше. А вот про рейтинги игроков сказать что-то однозначно нельзя, так как мы не имеем эталонный рейтинг сил игроков на данный момент

###### Скорее всего при расчете формул, либо их реализации, у меня могут быть ошибки. Думаю сделал все что мог на данный момент. Возможно стоило делать модели на основе линейной или логистической регрессии, но я поначалу не сообразил как, потом было поздно)